<a href="https://colab.research.google.com/github/arafatsyed/FindItAi/blob/main/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from  torchvision.datasets import ImageFolder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchvision import datasets

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

training_dataset = ImageFolder('/content/drive/MyDrive/APS360/new_training',transform=transform)
validation_dataset = ImageFolder('/content/drive/MyDrive/APS360/new_validation',transform=transform)
testing_dataset = ImageFolder('/content/drive/MyDrive/APS360/new_testing',transform=transform);


In [ ]:
print(training_dataset.class_to_idx)

{'dhaka': 0, 'johannesburg': 1, 'manhattan': 2, 'paris': 3, 'sao_paulo': 4, 'seoul': 5}


In [ ]:
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
res_net = torchvision.models.resnet50(pretrained=True)
res_net_conv = nn.Sequential(*list(res_net.children())[:-2])
res_net_conv.training = False

In [ ]:
from torchsummary import summary

summary(res_net_conv,input_size=(3,224,224))

In [ ]:
batch_size = 1
#Get data in batch size of 1 (out put 1x3x244x244)
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size,num_workers=1)
val_loader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size,num_workers=1)
test_loader = torch.utils.data.DataLoader(testing_dataset,batch_size=batch_size,num_workers=1)

for img,label in train_loader:
  print(img.shape)
  features = res_net_conv(img)
  print(features.shape)
  
  break


torch.Size([1, 3, 224, 224])
torch.Size([1, 2048, 7, 7])


In [ ]:
import os

batch_size = 1
#Get data in batch size of 1 (out put 1x3x244x244)
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size,num_workers=1)
val_loader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size,num_workers=1)
test_loader = torch.utils.data.DataLoader(testing_dataset,batch_size=batch_size,num_workers=1)

labels = ['dhaka','johannesburg','manhattan','paris','sao_paulo','seoul']

#location on my local drive
folderpath = "/content/drive/MyDrive/APS360/resnet_features"
train_dir = os.path.join(folderpath,'train/')
val_dir = os.path.join(folderpath,'val/')
test_dir = os.path.join(folderpath,'test/')

#function saves data as folders labeled A, B, C etc
#to use datasetFolder to easily get these tensors
def save_alexnet_featues(dataset,path):
  pic_id = 0
  for image,label in dataset:
    #features = alexnet.features(image)
    features = res_net_conv(image)
    #remove gradient
    features = torch.from_numpy(features.detach().numpy()).squeeze(0)
    torch.save(features, path + str(labels[label]) + '/' + str(pic_id) +'_'+str(labels[label]) + '.tensor')
    pic_id+=1


In [ ]:
#save_alexnet_featues(train_loader,train_dir)
#save_alexnet_featues(val_loader,val_dir)
#save_alexnet_featues(test_loader,test_dir)

In [ ]:
#load in tensors from alex net
training_dataset = torchvision.datasets.DatasetFolder('/content/drive/MyDrive/APS360/new_features/train',loader = torch.load, extensions=('.tensor'))
validation_dataset =  torchvision.datasets.DatasetFolder('/content/drive/MyDrive/APS360/new_features/val',loader = torch.load, extensions=('.tensor'))
testing_dataset =  torchvision.datasets.DatasetFolder('/content/drive/MyDrive/APS360/new_features/test',loader = torch.load, extensions=('.tensor'))

#load tensors from res_net
# training_dataset = torchvision.datasets.DatasetFolder('/content/drive/MyDrive/APS360/resnet_features/train',loader = torch.load, extensions=('.tensor'))
# validation_dataset =  torchvision.datasets.DatasetFolder('/content/drive/MyDrive/APS360/resnet_features/val',loader = torch.load, extensions=('.tensor'))
# testing_dataset =  torchvision.datasets.DatasetFolder('/content/drive/MyDrive/APS360/resnet_features/test',loader = torch.load, extensions=('.tensor'))

print(testing_dataset[0][1])
print(testing_dataset.class_to_idx)




0
{'dhaka': 0, 'johannesburg': 1, 'manhattan': 2, 'paris': 3, 'sao_paulo': 4, 'seoul': 5}


In [ ]:
class ResNetANNClassifier(nn.Module):
  def __init__(self):
    super(ResNetANNClassifier,self).__init__()
    self.name = "ResNetANN"
    self.fc1 = nn.Linear(2048*7*7,5000)
    self.fc2 = nn.Linear(5000,32)
    self.fc3 = nn.Linear(32,6)
  
    self.dropout = nn.Dropout(0.25)

  def forward(self,x):
    x = x.view(-1,2048*7*7)
    #x = self.dropout(x)
    x = F.relu(self.fc1(x))
   # x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = self.dropout(x)
    x = self.fc3(x)
    return x;



In [ ]:
class ANNClassifier(nn.Module):
  def __init__(self):
    super(ANNClassifier,self).__init__()
    self.name = "ANN"
    self.fc1 = nn.Linear(256*6*6,500)
    #self.fc2 = nn.Linear(500,32)
    self.fc3 = nn.Linear(500,6)

    self.dropout = nn.Dropout(0.15)

  def forward(self,x):
    x = x.view(-1,256*6*6)
    #x = self.dropout(x)
    x = F.relu(self.fc1(x))
   # x = self.dropout(x)
    #x = F.relu(self.fc2(x))
    #x = self.dropout(x)
    x = self.fc3(x)
    return x;



In [ ]:
class GestureNet(nn.Module):
  def __init__(self):
    super(GestureNet, self).__init__()
    self.name="gesture"
    self.conv1 = nn.Conv2d(3, 10, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(10, 10, 5)
    self.fc1 = nn.Linear(10 * 53 * 53, 500)
    self.fc2 = nn.Linear(500, 6)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(x.size(0), 10*53*53)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = x.squeeze(1) # Flatten to [batch_size]
    return x        



In [ ]:
use_cuda = True

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "/content/drive/MyDrive/APS360/models/model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

#takes the model, the dataset to test on, returns the accuracy on that dataset
def get_accuracy(net,data,train=False):

  labels = ['dhaka','johannesburg','manhattan','paris','sao_paulo','seoul']
  cor = 0
  total = 0
  total_class = np.zeros(6)
  total_correct = np.zeros(6)
  total_pred = np.zeros(6)
  for img,label in iter(data):
    if use_cuda and torch.cuda.is_available():
      img=img.cuda()
      label=label.cuda()
    output = net(img)
    pred=output.max(1,keepdim=True)[1]
    
    index = 0
    for i in pred:
      total_pred[i]+=1
      
      total_class[label[index]]+=1
      if(i==label[index]):
        total_correct[i]+=1
      index+=1
    #print(total_class,total_correct,total_pred)
    #return 1
    cor +=pred.eq(label.view_as(pred)).sum().item()
    total +=img.shape[0]
    #print(cor)
  print(total_class,total_correct,total_pred)
  return cor/total

def train_net(net, batch_size=64,learning_rate=0.01,num_epochs=30,custom=False):

  torch.manual_seed(1000)

  #shuffle data
  if(not custom):
    train_loader = torch.utils.data.DataLoader(training_dataset,batch_size=batch_size,shuffle=True)
    val_loader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size,shuffle=True)
  # else:
  #   train_loader = torch.utils.data.DataLoader(custom_img,batch_size=batch_size,shuffle=True)
  #   val_loader = torch.utils.data.DataLoader(validation_dataset,batch_size=512,shuffle=True)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(),lr=learning_rate,momentum=0.9)

  train_err = np.zeros(num_epochs)
  train_loss = np.zeros(num_epochs)

  train_acc = np.zeros(num_epochs)
  train_loss = np.zeros(num_epochs)
  val_acc = np.zeros(num_epochs)
  val_loss = np.zeros(num_epochs)  


  #iters,losses,train_acc,val_acc = [],[],[],[]
  epochs=[]
  n=0
  cor=0
  total=0
  print("starting training")
  for epoch in range(num_epochs):
    cor=0
    total=0
    total_train_loss = 0.0
    total_val_loss = 0.0
    i=0
    for inputs,labels in iter(train_loader):
     
      if use_cuda and torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()     

      output = net(inputs)
      loss=criterion(output,labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      
      #loss
      total_train_loss += loss.item()
      i+=1

      #mini batch accuracy
      pred = output.max(1,keepdim=True)[1]
      cor +=pred.eq(labels.view_as(pred)).sum().item()
      total+=labels.shape[0]

    train_acc[epoch]=(cor/total) 
    train_loss[epoch]=(float(total_train_loss)/(i+1))

    i=0
    cor=0
    total=0
    for inputs,labels in iter(val_loader):
      
      if use_cuda and torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()     
      #get output and calculate loss
      output = net(inputs)
      loss = criterion(output,labels)
      total_val_loss += loss.item()
      i+=1

      #calculate accuracy
      pred = output.max(1,keepdim=True)[1]
      cor +=pred.eq(labels.view_as(pred)).sum().item()
      total+=labels.shape[0]
    
    val_acc[epoch]=(cor/total)
    val_loss[epoch]=(float(total_val_loss)/(i+1))

    epochs.append(epoch)
    print("Epoch: %d" % epoch,"Training Acc: %6.2f"% train_acc[epoch],"Validation Acc: %6.2f"% val_acc[epoch])
  
  #Save model
  model_path = get_model_name(net.name,batch_size,learning_rate,epoch)
  torch.save(net.state_dict(),model_path)

  plt.title("Training Accuracy vs Validation Accuracy")
  plt.plot(epochs,train_acc,label="Training Acc")
  plt.plot(epochs,val_acc,label="Validation Acc")
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.show()

  plt.title("Training Loss vs Validation Loss")
  plt.plot(epochs,train_loss,label="Training Loss")
  plt.plot(epochs,val_loss,label="Val Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.show()

  print("Final Training Acc: %6.2f"% train_acc[-1],"Final Validation Acc: %6.2f"% val_acc[-1])

In [ ]:
first_net = ANNClassifier()

if use_cuda and torch.cuda.is_available():
  first_net.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

train_net(first_net,128,0.0003,40)

In [ ]:
first_net = ANNClassifier()

if use_cuda and torch.cuda.is_available():
  first_net.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

state = torch.load('/content/drive/MyDrive/APS360/models/model_ANN_bs128_lr0.0003_epoch39')
first_net.load_state_dict(state)

CUDA is available!  Training on GPU ...


<All keys matched successfully>

In [ ]:
test_loader = torch.utils.data.DataLoader(testing_dataset,batch_size=64,shuffle=True)

print("Testing Accuracy: %6.2f"% get_accuracy(first_net,test_loader))

Testing Accuracy:   0.68


In [ ]:
train_loader = torch.utils.data.DataLoader(training_dataset,batch_size=64,shuffle=True)
print("Testing Accuracy: %6.2f"% get_accuracy(first_net,test_loader))

In [ ]:
second_net = GestureNet()

if use_cuda and torch.cuda.is_available():
  second_net.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

train_net(second_net,128,0.008,30)

In [ ]:
resnet_training = ResNetANNClassifier()

if use_cuda and torch.cuda.is_available():
  resnet_training.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

train_net(resnet_training,64,0.0001,35)

In [ ]:
resnet_training = ResNetANNClassifier()

if use_cuda and torch.cuda.is_available():
  resnet_training.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

train_net(resnet_training,256,0.001,35)